# 1. Improving credit card scoring through data quality

We are all familiar with the axiom "Garbage in, garbage out", and this is very much true, specially in a setting and market where we see models getting more and more commmoditized. The business advatage will remain in the component that it is unique to every organization - the data.

In every case - and particularly for credit scoring use cases - data preparation is a paramount. Nevertheless, and althoug the achievments we have observed in the past few years, data preparation is still the most challenging and time-consuming step. Ensuring data quality helps data teams to achieve bigger ROI from AI initiatives at a fraction of the effort it used to, translating into better scorecards that positively impact the business and customer experience.

When we look into the credit scoring, there are particular issues that can dampen model accuracy - presence of outliers, missing values and the presence of imbalanced classes.

In this usecase we will explore not only an iterative, traceable and comparable data processing for to improve the quality of the data for credit scorecards, but also how to mitigate each one of the identified challenges: missing data, presence of duplicates and last but not the least, imabalanced data.


The dataset leveraged for the use-case can be easily found in [Kaggle - Give me some credit](https://www.kaggle.com/competitions/GiveMeSomeCredit/overview).

## Read the data

The first step is to read the data. As Data Scientists it is usual to find the use of CSV as the main data sources. For that reason we enable a flexible interface to consume CSV data.

- Through YData SDK - LocalConnector - this method returns automatically a Dataset object that is ready to be consumed in downstream applications such as Synthetisizer.
- Leveraging pandas - Pandas is available in YData image. This method requires the data to be converted into Dataset, if the user wants to leverage downstream applications such as Synthesizers, or the scale from our distributed computing engine.

In this particular example, we have decided to load the data while using pandas SDK method `read_csv`.

### Import needed packages

In [5]:
import os

import numpy as np
import pandas as pd

from ydata.utils.formats import read_json
from ydata.dataset import Dataset
from ydata.metadata import Metadata
from ydata.labs.datasources import DataSources

from ydata.connectors.filetype import FileType
from ydata.connectors import GCSConnector


from ydata.dataset.holdout import Holdout

### Read the data from an existing source

In [10]:
dataset = DataSources.get(uid='{datasource-id}').read()

metadata = Metadata(dataset)

/opt/conda/lib/python3.8/site-packages/ydata/metadata/metadata.py:143: UserWarning: Datasets other than Timeseries don't make use of dataset_attrs
  warn("Datasets other than Timeseries don't make use of dataset_attrs")


In [8]:
print(dataset)

Dataset 
 
Shape: (50000, 11)
Schema: 
                                  Column Variable type
0                       SeriousDlqin2yrs           int
1   RevolvingUtilizationOfUnsecuredLines         float
2                                    age           int
3   NumberOfTime30-59DaysPastDueNotWorse           int
4                              DebtRatio         float
5                          MonthlyIncome         float
6        NumberOfOpenCreditLinesAndLoans           int
7                NumberOfTimes90DaysLate           int
8           NumberRealEstateLoansOrLines           int
9   NumberOfTime60-89DaysPastDueNotWorse           int
10                    NumberOfDependents         float




In [9]:
metadata.update_datatypes({'SeriousDlqin2yrs': 'categorical'})

## Preparing the data for the data preparation workflow
Now that we have our dataset ready to start working, before any data analysis and preparation it is always recommended to create a holdout. This ensures unbiased results in terms of improvements achieved.

In [ ]:
holdout = Holdout()
train, test = holdout.get_split(dataset, metadata)

In [ ]:
metadata = Metadata(train)

In [ ]:
metadata.update_datatypes({'SeriousDlqin2yrs':'categorical'})

In [ ]:
print(dataset)

In [ ]:
#Get metadata warnings
warnings=[]
for warning, val in metadata.warnings.items():
    for col in val:
        try:
            level = col.details['level'].name
            value = round(col.details['value'], 4)
        except:
            level = None
            value = None
        warnings.append({'warning': warning, 'column': col.column, 'level': level, 'value':value})

df_warnings= pd.DataFrame(warnings)

## Create pipeline outputs
Setting our pipeline outputs to share data and artifacts in-between the different steps of the pipeline, as well as for visualization.

### Outputs

In [ ]:
#Save the train metadata
metadata.save('metadata.pkl')

#Save the train dataset
train.to_pandas().to_csv('train.csv')

#Save the holdout set
test.to_pandas().to_csv('test.csv')

### Visualizations

In [11]:
import json

metadata = {
    'outputs' : [
        {
      'type': 'table',
      'storage': 'inline',
      'format': 'csv',
      'header': list(df_warnings.columns),
      'source': df_warnings.to_csv(header=False, index=False)
    }
    ]
  }

with open('mlpipeline-ui-metadata.json', 'w') as metadata_file:
    json.dump(metadata, metadata_file)